## Imports

In [1]:
import os, sys
import numpy as np
import matplotlib
import matplotlib.pyplot as plt

import tensorflow as tf
import pandas as pd

from object_detection.utils import label_map_util
from object_detection.utils import dataset_util
tf.compat.v1.enable_eager_execution()

In [2]:
#
# Ammend these variables to your own setup
#

BASE_PATH      = "C:\\Users\\Thomas.Jubb\\Documents\\tensorflow_detector\\training_demo\\"
PATH_TO_IMAGES = BASE_PATH + "images"
PATH_TO_LABELS = BASE_PATH + "annotations\item_map.pbtxt"

test_fraction = 0.1

## Read Data and Convert to JPG

The name of the game here is to re-jig the data in the 'bags' and 'df' variables into a format that Tensorflow likes.

In [3]:
#
# Read in the class labels for the images
#

df = pd.read_csv("images/labels.txt", header=None, names=["File", "xmin", "ymin", "xmax", "ymax", "Label"])
df

,File,xmin,ymin,xmax,ymax,Label
0,image_0.jpg,105,169,379,448,Coins
1,image_1.jpg,38,135,387,467,Coins


In [4]:
#
# Gather all the image data together
#

images_and_bboxes = []

for idx, obj in df.iterrows():
    box_coords = [obj["xmin"],obj["ymin"],obj["xmax"],obj["ymax"]]
    filename = obj["File"]
    
    images_and_bboxes.append({"filename":filename,
                              "box_coords":box_coords, 
                              "class":obj["Label"]})

In [5]:
#
# Create the mapping from class name -> index
#

category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

class_to_index = {}
for key, value in category_index.items():
    class_to_index[value["name"]] = key

# Creating Record Files

In [6]:
def create_tf_example(example, path, class_mapping):
    """
    Create a single Tensorflow Example object to be used in creating record
    
    Parameters
    ----------
    
        example : dict
            A single object; the dictionary should contains the keys "filename" referring to the jpg containing
            the object, and "box_coords" which gives the location of the object, and "class" the name of the object
            
        path : str
            The path to the image files.
    
    Returns
    -------
    
        The tf Example object
    
    """
    path = (path + os.sep).encode('ascii')
    filename = example['filename'].encode('ascii')
    image_format = b'jpg'
    
    image = plt.imread(path +filename, "jpg")
    
    height, width = image.shape[:2]
    
    # Encode the jpg to byte form
    with tf.gfile.GFile(path+filename, 'rb') as fid:
        encoded_jpg = bytes(fid.read())

    # normalize the box coordinates
    xmins = [example['box_coords'][0]/width] 
    ymins = [example['box_coords'][1]/height]
    xmaxs = [example['box_coords'][2]/width] 
    ymaxs = [example['box_coords'][3]/height]

    classes_text = [example["class"].encode('ascii')]
    classes      = [class_mapping[example["class"]]]

    # create the example
    tf_example = tf.train.Example(features=tf.train.Features(feature={
      'image/height'             : dataset_util.int64_feature(height),
      'image/width'              : dataset_util.int64_feature(width),
      'image/filename'           : dataset_util.bytes_feature(filename),
      'image/source_id'          : dataset_util.bytes_feature(filename),
      'image/encoded'            : dataset_util.bytes_feature(encoded_jpg),
      'image/format'             : dataset_util.bytes_feature(image_format),
      'image/object/bbox/xmin'   : dataset_util.float_list_feature(xmins),
      'image/object/bbox/xmax'   : dataset_util.float_list_feature(xmaxs),
      'image/object/bbox/ymin'   : dataset_util.float_list_feature(ymins),
      'image/object/bbox/ymax'   : dataset_util.float_list_feature(ymaxs),
      'image/object/class/text'  : dataset_util.bytes_list_feature(classes_text),
      'image/object/class/label' : dataset_util.int64_list_feature(classes),
      }))
    return tf_example

In [7]:
def _create_record_file_path(path):
    # clear the otuput path flag and set
    flags = tf.app.flags
    FLAGS = flags.FLAGS
    try:
        FLAGS.__delattr__('output_path')
        FLAGS.__delattr__('f')
    except:
        pass

    tf.app.flags.DEFINE_string('f', '', 'kernel') 
    flags.DEFINE_string('output_path', path, '')
    FLAGS = flags.FLAGS

    print("New record file : {}".format(flags.FLAGS.output_path))  

In [8]:
def _create_record_file(path, examples):
    _create_record_file_path(path)
    FLAGS = tf.app.flags.FLAGS
    
    writer = tf.python_io.TFRecordWriter(FLAGS.output_path)
    for example in examples:
        tf_example = create_tf_example(example, PATH_TO_IMAGES, class_to_index)
        writer.write(tf_example.SerializeToString())
    writer.close()
    
    print("Wrote {} examples".format(len(examples)))


In [9]:
#
# Train test split
#

indices = np.random.permutation(len(images_and_bboxes))
cut = int(np.ceil(len(images_and_bboxes)* test_fraction))
train_examples = images_and_bboxes[cut:]
test_examples  = images_and_bboxes[:cut]

_create_record_file(BASE_PATH + r'annotations\train.record', train_examples)
_create_record_file(BASE_PATH + r'annotations\test.record', test_examples)

New record file : C:\Users\Thomas.Jubb\Documents\tensorflow_detector\training_demo\annotations\train.record
Wrote 1 examples
New record file : C:\Users\Thomas.Jubb\Documents\tensorflow_detector\training_demo\annotations\test.record
Wrote 1 examples


### Test Read Example

In [10]:
filenames = [BASE_PATH + r'annotations\train.record']
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset
tf.executing_eagerly()
for raw_record in raw_dataset.take(10):
    print(repr(raw_record))

<tf.Tensor: id=20, shape=(), dtype=string, numpy=b'\n\x97\x80\x01\n\xac}\n\rimage/encoded\x12\x9a}\n\x97}\n\x94}\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00d\x00d\x00\x00\xff\xdb\x00C\x00\x08\x06\x06\x07\x06\x05\x08\x07\x07\x07\t\t\x08\n\x0c\x14\r\x0c\x0b\x0b\x0c\x19\x12\x13\x0f\x14\x1d\x1a\x1f\x1e\x1d\x1a\x1c\x1c $.\' ",#\x1c\x1c(7),01444\x1f\'9=82<.342\xff\xdb\x00C\x01\t\t\t\x0c\x0b\x0c\x18\r\r\x182!\x1c!22222222222222222222222222222222222222222222222222\xff\xc0\x00\x11\x08\x02@\x02\x00\x03\x01"\x00\x02\x11\x01\x03\x11\x01\xff\xc4\x00\x1f\x00\x00\x01\x05\x01\x01\x01\x01\x01\x01\x00\x00\x00\x00\x00\x00\x00\x00\x01\x02\x03\x04\x05\x06\x07\x08\t\n\x0b\xff\xc4\x00\xb5\x10\x00\x02\x01\x03\x03\x02\x04\x03\x05\x05\x04\x04\x00\x00\x01}\x01\x02\x03\x00\x04\x11\x05\x12!1A\x06\x13Qa\x07"q\x142\x81\x91\xa1\x08#B\xb1\xc1\x15R\xd1\xf0$3br\x82\t\n\x16\x17\x18\x19\x1a%&\'()*456789:CDEFGHIJSTUVWXYZcdefghijstuvwxyz\x83\x84\x85\x86\x87\x88\x89\x8a\x92\x93\x94\x95\x96\x97\x98\x99\x9a\xa2\xa3\xa4\xa5